In [1]:
# 8th July, 2019
#Author Esha Sarkar
import pandas
import numpy as np
import keras

from random import seed
from random import randint
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras import optimizers
from random import seed
from random import randint
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import LSTM,Dropout,Reshape
from math import sqrt
from sklearn.metrics import mean_squared_error
import tensorflow as tf

Using TensorFlow backend.


In [2]:
TagGenotypes_1k = np.array(pandas.read_csv('sorted_tag_SNPs_1k_genotypes.txt', sep='\s+', delimiter=None, header=None, names=None, index_col=None))
TargetGenotypes = np.array(pandas.read_csv('sorted_target_SNP_genotypes.txt', sep='\s+', delimiter=None, header=None, names=None, index_col=None))

In [3]:
DatasetX1_1k = TagGenotypes_1k[:,4:].transpose()
DatasetY1_1k = TargetGenotypes[:,4:].transpose()
ShufflingVector_1k = np.random.permutation(DatasetX1_1k.shape[0])
DatasetX_1k = DatasetX1_1k[ShufflingVector_1k]
DatasetY_1k = DatasetY1_1k[ShufflingVector_1k]

In [4]:
def OneHotEncoder(Matrix):
    Temp = np.zeros((Matrix.shape[0],Matrix.shape[1],3))
    for num, i in enumerate(Matrix):
        Temp[num] = (tf.keras.utils.to_categorical(i,3))
    return (Temp)

In [5]:
split = int(0.8*(DatasetX_1k.shape[0]))
Xtr1_1k = DatasetX_1k[:split]
Xtr_1k = (OneHotEncoder(Xtr1_1k)).reshape((Xtr1_1k.shape[0],Xtr1_1k.shape[1]*3))

Xts1_1k = DatasetX_1k[split:]
Xts_1k = (OneHotEncoder(Xts1_1k)).reshape((Xts1_1k.shape[0],Xts1_1k.shape[1]*3))

ytr1_1k = DatasetY_1k[:split]
ytr_1k = (OneHotEncoder(ytr1_1k)).reshape((ytr1_1k.shape[0],ytr1_1k.shape[1]*3))

yts1_1k = DatasetY_1k[split:]
yts_1k = (OneHotEncoder(yts1_1k)).reshape((yts1_1k.shape[0],yts1_1k.shape[1]*3))

In [6]:
np.savetxt('XTrain_1k.txt', Xtr1_1k, delimiter=',',fmt='%1d') 
np.savetxt('XTest_1k.txt', Xts1_1k, delimiter=',',fmt='%1d') 
np.savetxt('YTrain_1k.txt', ytr1_1k, delimiter=',',fmt='%1d') 
np.savetxt('YTest_1k.txt', yts1_1k, delimiter=',',fmt='%1d') 

In [7]:
def Accuracy(X,y,ModelName):
    y_actual = np.argmax(y.reshape((y.shape[0],int(y.shape[1]/3),3)),2)
    y1 = (ModelName.predict(X))
    y_hat = np.argmax((y1.reshape((y1.shape[0],int(y1.shape[1]/3),3))),2)
    A = (np.count_nonzero(y_hat==y_actual))/(y_actual.shape[0]*y_actual.shape[1])
    return (A)

In [8]:
TagNos = DatasetX_1k.shape[1]
from keras.constraints import maxnorm, nonneg
from keras import backend as K
K.clear_session()
model_1k = Sequential()
model_1k.add(Dense(1500, input_dim=TagNos*3,bias_constraint=keras.constraints.NonNeg(),kernel_constraint=keras.constraints.NonNeg()))
opt = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_1k.compile(loss='mean_squared_error', optimizer=opt)
model_1k.summary()  

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              43858500  
Total params: 43,858,500
Trainable params: 43,858,500
Non-trainable params: 0
_________________________________________________________________


In [10]:
kFolds = 5
kFoldAccuracy = []
split_value = int((1/kFolds)*Xtr_1k.shape[0])
for fold in range(kFolds):
    Testfrom = fold*split_value
    Testto = (fold+1)*split_value
    
    Trainto1 = (fold)*split_value    
    Trainfrom2 = (fold+1)*split_value
    print ("========================================================================================================")
    print ("KFOLD: ",fold)
    print ("Test:", Testfrom,Testto)
    print ("Train: [",0,Trainto1,"] [",Testto,Xtr_1k.shape[0],"]")
    
    X_train = ((np.concatenate((Xtr_1k[:Trainto1],Xtr_1k[Trainfrom2:])))).astype('float')
    X_ts = (Xtr_1k[Testfrom:Testto]).astype('float')

    y_train = ((np.concatenate((ytr_1k[:Trainto1],ytr_1k[Trainfrom2:])))/2).astype('float')
    y_ts = (ytr_1k[Testfrom:Testto]).astype('float')
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                              patience=3, min_lr=0.0001)
    model_1k.fit(X_train, y_train, epochs=30, verbose=1,callbacks=[reduce_lr])
    a=Accuracy(X_train,y_train,model_1k)
    aprime=Accuracy(X_ts,y_ts,model_1k)
    print ("Training accuracy: ", np.mean(a))
    print ("Test: ", np.mean(aprime))
    kFoldAccuracy.append(np.mean(aprime))
    print ("========================================================================================================")

KFOLD:  0
Test: 0 400
Train: [ 0 0 ] [ 400 2003 ]
Epoch 1/30
1603/1603 [==============================] - 44s 28ms/step - loss: 0.0112
Epoch 2/30
1603/1603 [==============================] - 46s 29ms/step - loss: 0.0109
Epoch 3/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0106
Epoch 4/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0103
Epoch 5/30
1603/1603 [==============================] - 46s 29ms/step - loss: 0.0100
Epoch 6/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0098
Epoch 7/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0096
Epoch 8/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0093
Epoch 9/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0091
Epoch 10/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0089
Epoch 11/30
1603/1603 [==============================] - 44s 27ms/step - loss: 0.0087
Epoch 12/30
1

1603/1603 [==============================] - 46s 28ms/step - loss: 0.0050
Epoch 28/30
1603/1603 [==============================] - 48s 30ms/step - loss: 0.0050
Epoch 29/30
1603/1603 [==============================] - 49s 30ms/step - loss: 0.0050
Epoch 30/30
1603/1603 [==============================] - 46s 29ms/step - loss: 0.0050
Training accuracy:  0.9752813474734872
Test:  0.96634
KFOLD:  3
Test: 1200 1600
Train: [ 0 1200 ] [ 1600 2003 ]
Epoch 1/30
1603/1603 [==============================] - 45s 28ms/step - loss: 0.0053
Epoch 2/30
1603/1603 [==============================] - 48s 30ms/step - loss: 0.0053
Epoch 3/30
1603/1603 [==============================] - 47s 29ms/step - loss: 0.0052
Epoch 4/30
1603/1603 [==============================] - 48s 30ms/step - loss: 0.0052
Epoch 5/30
1603/1603 [==============================] - 46s 29ms/step - loss: 0.0052
Epoch 6/30
1603/1603 [==============================] - 47s 30ms/step - loss: 0.0052
Epoch 7/30
1603/1603 [========================

In [11]:
Accuracy(Xts_1k,yts_1k,model_1k)

0.9593892215568862

In [ ]:
Accuracy(Xtr_1k,ytr_1k,model_10k)

In [12]:
Layers_1k = model_1k.layers
W_1k,b_1k = Layers_1k[0].get_weights()
W_1k[W_1k==-0.0]=0.0
b_1k[b_1k==-0.0]=0.0

In [13]:
np.savetxt('W_1k_complete.txt', W_1k, delimiter=',',fmt='%1.4f') 

In [ ]:
Layers_10k = model_10k.layers
W_10k,b_10k = Layers_10k[0].get_weights()
W_10k[W_10k==-0.0]=0.0
b_10k[b_10k==-0.0]=0.0

In [ ]:
b[b==-0.0]=0.0

In [ ]:
np.savetxt('W_10k_complete.txt', W_10k, delimiter=',',fmt='%1.4f') 
np.savetxt('b_10k_complete.txt', b_10k, delimiter=',',fmt='%1.4f') 

In [ ]:
y1 = model_10k.predict(Xts_10k)
yts_10k_hat = np.argmax((y1.reshape((y1.shape[0],int(y1.shape[1]/3),3))),2)

In [ ]:
O = np.dot(Xts_10k,W)+b
O_num = np.argmax((O.reshape((O.shape[0],int(O.shape[1]/3),3))),2)

In [ ]:
for u in O_num==yts_10k_hat:
    print (True in u)

In [ ]:
def scale(Matrix):
    min_value = np.min(Matrix)
    max_value = np.max(Matrix)
    ScaledMatrix = (Matrix-min_value)/(max_value-min_value)
    return (ScaledMatrix)

In [ ]:
yts_hat_normalized = scale(model.predict(Xts))

In [ ]:
temp = yts_hat_normalized.reshape((500,500,3))

In [ ]:
divs = np.array([np.sum(i,1) for i in temp])

In [ ]:
yts_preds = np.zeros((temp.shape))
for i in range(500):
    for j in range(500):
        yts_preds[i][j] = temp[i][j]/divs[i][j]

In [ ]:
temp2 = yts_preds.reshape((250000,3))

In [ ]:
temp2

In [ ]:
yts_print = yts.reshape((500,500,3)) 

In [ ]:
yts.shape

In [ ]:
np.savetxt('y_pred.txt', temp2, delimiter=',',fmt='%1.6f') 

In [ ]:
np.savetxt('y_actual.txt', yts1.astype('int').reshape((250000,1)), delimiter=',',fmt='%1d') 

## Homomorphic part

In [ ]:
W = np.loadtxt('W.txt',delimiter=',')
b = np.loadtxt('b.txt',delimiter=',')

In [ ]:
import time
start_time = time.time()
np.dot(Xts,W)+b
print ("Time required in plain-text",time.time()-start_time)

In [ ]:
from phe import paillier
puk, prk = paillier.generate_paillier_keypair(n_length=1024)

In [ ]:
Xts_10k.shape

In [ ]:
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)

In [ ]:
xtemp1 = Xts.reshape((500*3135,1))

In [ ]:
pool.apply([puk.encrypt(i) for i in range(5)])

In [ ]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
Xts_e1 = pool.apply(EncryptVector(puk,a))
print (time.time()-start_time)

In [ ]:
def EncryptVector(pubkey, x):
    xtemp = x.reshape((x.shape[0]*x.shape[1],1))
    return (np.array([pubkey.encrypt(xtemp[i][0]) for i in range(xtemp.shape[0])])).reshape(x.shape)

def DecryptVector(privkey, x):
    xtemp = x.reshape((x.shape[0]*x.shape[1],1))
    return (np.array([privkey.decrypt(i[0]) for i in xtemp])).reshape(x.shape)

def AddEncryptedVectors(x, y):

    if len(x) != len(y):
        raise Exception('Encrypted vectors must have the same size')

    return [x[i] + y[i] for i in range(len(x))]

In [ ]:
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)

In [ ]:
start_time = time.time()
d = b+c_e
print (time.time()-start_time)

In [ ]:
Check_Xts = DecryptVector(prk,Xts_e)

In [ ]:
W.shape

In [ ]:
t1 = np.ones((10,3135))
t2 = np.ones((3135,1500))
t1_e = EncryptVector(puk,t1)
#t2_e = EncryptVector(puk,t2)

In [ ]:
start_time = time.time()
m_e = np.dot(t1_e,t2)
print (time.time()-start_time)

In [ ]:
start_time = time.time()
m_e = np.dot(t1_e,t2)
print (time.time()-start_time)

In [ ]:
m = DecryptVector(prk,m_e)

In [ ]:
a = (np.ones((500,1500)))*1.32
b = (np.ones((500,1500)))*2.32
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)
#b_e = EncryptVector(puk,b)
#c_e = a_e + b
#c = DecryptVector(c_e)

In [ ]:
start_time = time.time()
Xts_e.dot(W)
print (time.time()-start_time)

In [ ]:
g = DecryptVector(prk,f)

In [ ]:
g

In [ ]:



def decrypt_vector(privkey, x):
    return np.array([privkey.decrypt(i) for i in x])


def sum_encrypted_vectors(x, y):

    if len(x) != len(y):
        raise Exception('Encrypted vectors must have the same size')

    return [x[i] + y[i] for i in range(len(x))]

In [ ]:
Xts.shape

In [ ]:
EncXts = []
for i in range(Xts.shape[0]):
    for j in range(Xts.shape[1]):
        #print (i,j)
        EncXts[i][j]=(puk.encrypt(Xts[i][j])).ciphertext()

In [ ]:
EncXts2 = []
for i in range(Xts.shape[0]):
    for j in range(Xts.shape[1]):
        #print (i,j)
        EncXts2.append(puk.encrypt(Xts[i][j]))

In [ ]:
EncXts2 = np.array(EncXts2)

In [ ]:
EncXts2 = EncXts2.reshape((Xts.shape))

In [ ]:
a = np.array([[9,5],[2,5],[1,7]])

In [ ]:
b = np.array([[puk.encrypt(6)],[puk.encrypt(11)]])
d = np.array([[6],[11]])

In [ ]:
f = np.array([[puk.encrypt(8)],[puk.encrypt(1)]])
g = np.array([[8],[1]])

In [ ]:
def MatMul(A,B):
    ResultsTemp = []
    for i in range(A.shape[0]):
        #print (i)
        for j in range(B.shape[1]):
            temp = 0
            for k in range(B.shape[0]):
                temp += A[i][k] * B[k][j]
            ResultsTemp.append(temp)
    ResultsTemp = np.array(ResultsTemp)
    Results = ResultsTemp.reshape((A.shape[0],B.shape[1]))
    return (Results)

In [ ]:
z = b+f

In [ ]:
W.shape

In [ ]:
import time
start_time = time.time()
e = MatMul(a,b)
print ("Time required to multiply",time.time()-start_time)

In [ ]:
for i in z:
    print (prk.decrypt(i[0]))

In [ ]:
e

In [ ]:
O = np.argmax((np.matmul(Xts,W)+b).reshape((504,500,3)),2)

In [ ]:
O2 = np.argmax((model.predict(Xts)).reshape((504,500,3)),2)

In [ ]:
np.count_nonzero((O!=O2))

In [ ]:
np.savetxt('W_10k.txt', W, delimiter=',') 

In [ ]:
np.savetxt('b_10k.txt',b,delimiter=',')

In [ ]:
def scale(Matrix):
    min_value = np.min(Matrix)
    max_value = np.max(Matrix)
    ScaledMatrix = (Matrix-min_value)/(max_value-min_value)
    return (ScaledMatrix)